<a href="https://colab.research.google.com/github/jeongwoo22/ESAA_assignments/blob/main/OB_%EA%B3%BC%EC%A0%9C10_1031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 분류 실습 - 20 뉴스그룹 분류 
- 파이썬 머신러닝 완벽 가이드 487~496

- 텍스트 분류 : 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 하습 모델을 이용해 다른 문서의 분류를 예측
- 텍스트 ==> 피처 벡터화로 변환하면 **희소행렬 형태가 된다** 
- 희소 행렬에 분류를 잘하는 것 : 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등
- 텍스트를 기반으로 분류를 수행할 때는 먼저 텍스트를 정규화한 뒤 피처 벡터화를 적용 ==> 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가 

## 텍스트 정규화

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)

In [3]:
# fetch_20~ 은 사이킷런의 다른 데이터 세트 예제와 같이 파이썬 딕셔너리와 유사한 bunch 객체를 반환
news_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

- 파일 네임에 key 이름이 눈에 띔 ==> 패키지 api 가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명을 지칭

In [4]:
print("target 클래스의 이름")
print(news_data.target_names)

target 클래스의 이름
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


- 클래스는 0~ 19까지 20개

In [5]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 개별 텍스트 데이터는 기사 내용과 더불어 작성자, 소속, 이메일 등 다양한 정보가 있다.
- 제목, 소속 등 정보는 뉴스그룹 분류의 target과 유사한 데이터를 가진 경우가 많기에 내용을 제외한 정보는 제거한다.
- 순수한 텍스트만으로 어떤 뉴스그룹에 속하는지 확인한다.

In [6]:
# train - 내용 외 정보 제거
train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

<class 'list'>


In [7]:
# test - 내용 외 정보 제거
test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target

- 데이터를 불러올 때 remove로 헤더, 푸터 등의 정보를 제거하여 불러왔다.

- subset을 이용해서 train, test도 분리하였다.

- 11,314개의 뉴스그룹 문서가 리스트 형태로 train, 7,532개가 test로 주어졌다.

## 피처 벡터화 변환과 머신러닝 모델 학습/ 예측 /평가 

- 학습 데이터 : 11314개의 뉴스그룹 문서 (리스트)
- 테스트 데이터 : 7532개의 문서 (리스트)
- countvectorizer 로 학습 데이터 텍스트를 피처 벡터화 함 ==> 이 때 테스트 데이터 셋도 피처 벡터화 할 때 반드시 학습 데이터를 이용애 fit 이 수행된 countvecterizer  객체로 데이터 변환 해야함 ==> 그래야 피처 개수가 같아짐. 
- ++ fit_transform() 을 사용하면 안됨. => 피처 개수가 달라짐

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
X_test_cnt_vect = cnt_vect.transform(X_test)

print(f"학습 데이터 Text의 CountVectorizer Shape: {X_train_cnt_vect.shape}")

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


- 피처 벡터화로 피처를 추출한 결과 101,631개의 피처(단어)가 만들어졌다.

- 피처 벡터화시 반드시 train으로 학습한 벡터화 객체로 test를 벡터화 하여야한다.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 로지스틱 회귀
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
lr_acc = accuracy_score(y_test, pred)

print(f"CountVectorized Logistic Regression 예측 정확도: {lr_acc:.3f}")

CountVectorized Logistic Regression 예측 정확도: 0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


- Count 벡터화 후 로지스틱 회귀분석의 예측 정확도 :  0.604

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화 후 로지스틱 회귀분석을 진행
# TF-IDF Vectorization - train
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)

# TF-IDF Vectorization - test
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# 로지스틱 회귀
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
lr_acc = accuracy_score(y_test, pred)

print(f"TF-IDF Logistic Regression 예측 정확도: {lr_acc:.3f}")

TF-IDF Logistic Regression 예측 정확도: 0.674


- 예측 정확도가 0.674로 Count 벡터화에 비해 높게 나타났다.

- 일반적으로 문서 내에 텍스트가 많고, 문서가 많은 경우 TF-IDF 벡터화가 더 좋은 결과를 도출한다.

In [11]:
# 피처 벡터화의 파라미터 조정
# TF-IDF Vectorization - train
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)

# TF-IDF Vectorization - test
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# 로지스틱 회귀
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
lr_acc = accuracy_score(y_test, pred)

print(f"TF-IDF 파라미터 조정 후 예측 정확도: {lr_acc:.3f}")

TF-IDF 파라미터 조정 후 예측 정확도: 0.692


- 피처 벡터화에서 stop_words로 영어를 지정하여 영어 스톱 워드는 제외한다.

- ngram_range로 단어를 2개씩 묶어서 피처로 추출한다.

- max_df로 전체 문서를 통틀어 300번 이상 존재하는 단어는 제외한다.

- 앞서 파라미터를 디폴트로 설정하였을 때에 비해 예측 정확도가 증가하였다.

In [12]:
from sklearn.model_selection import GridSearchCV

# 최적 하이퍼 파라미터: C
params = {
    "C":[0.01, 0.1, 1, 5, 10]
}

# GridSearchCV
grid_cv_lr = GridSearchCV(lr_clf ,param_grid = params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측/평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
lr_acc = accuracy_score(y_test, pred)

print(f"로지스틱 최적 하이퍼 파라미터 적용 후 예측 정확도: {lr_acc:.3f}")

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression best C parameter : {'C': 10}
로지스틱 최적 하이퍼 파라미터 적용 후 예측 정확도: 0.701


- GridSearchCV를 이용하여 로지스틱 최적 하이퍼 파라미터를 찾았다.

- 예측 정확도는 0.701으로 조금 상승하였다.

- 단점 : 수행 시간이 매우 오래 걸린다.

## 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV 와의 결합

In [14]:
from sklearn.pipeline import Pipeline

# TF-IDF Vectorization 객체, LogisticRegression 객체 파이프라인 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# TF-IDF Vectorization, LogisticRegression 한번에 수행
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
lr_acc = accuracy_score(y_test, pred)

print(f"Pipeline을 통한 Logistic Regression 예측 정확도: {lr_acc:.3f}")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline을 통한 Logistic Regression 예측 정확도: 0.701


- pipeline()은 전처리와 ML 학습을 한번에 수행 가능하다.

- 피처 벡터화 결과를 별도 데이터로 저장하지 않고 바로 ML 데이터로 입력되어 수행시간 절약이 가능하다.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Pipeline
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline 각각의 객체의 파라미터를 입력
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}

# GridSearchCV: pipeline 객체를 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid = params, cv=3 , scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
lr_acc = accuracy_score(y_test, pred)

print(f'Pipeline을 통한 Logistic Regression 의 예측 정확도: {lr_acc:.3f}')

- pipeline()으로 GridSearchCV의 하이퍼 파라미터 튜닝이 가능하다.

- 파라미터는 각 객체명 + __ + 파라미터로 입력한다.

- GridSearchCV 외에도 5.4.6 최적 정규화의 다항회귀의 차수 결정 등에서 이러한 방법을 적용했었다.

- 텍스트 분류에는 로지스틱 외에도 서포트 벡터머신이나 나이브 베이즈 알고리즘을 자주 사용한다.